In [9]:
import numpy as np
import pandas as pd
import nltk
# nltk.download()

In [10]:
# loading the file
filename = '../dataset/Reviews only (Sentiment).csv'
file = open(filename, 'rt')
text = file.read()
file.close()

In [11]:
# Filter Out Punctuation

# split into words
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)

In [12]:
# remove all tokens that are not alphabetic
# words = [word for word in tokens if word.isalpha()]
# print(words[:100])

In [13]:
# Filter out Stop Words (and Pipeline)

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [14]:
# convert to lower case
tokens = [w.lower() for w in tokens]

In [15]:
# remove punctuation from each word
import string
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]

In [16]:
# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]

In [17]:
# filter out stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
print(words[:100])

['reviewtext', 'using', 'year', 'basically', 'quiet', 'environment', 'sound', 'pretty', 'good', 'nt', 'good', 'similarlypriced', 'studioclass', 'headsets', 'paying', 'noise', 'cancelling', 'raw', 'capability', 'pretty', 'decent', 'acceptable', 'detail', 'treble', 'vocal', 'bass', 'ranges', 'noise', 'cancelling', 'works', 'pretty', 'well', 'taken', 'several', 'intercontinental', 'flights', 'decent', 'job', 'blocking', 'engine', 'rumble', 'screaming', 'kids', 'also', 'decent', 'job', 'blocking', 'adult', 'human', 'voices', 'nice', 'office', 'however', 'must', 'pretty', 'big', 'ears', 'found', 'foam', 'cups', 'nt', 'quite', 'thick', 'enough', 'adequately', 'push', 'driver', 'away', 'outer', 'ear', 'hours', 'use', 'ears', 'start', 'ache', 'bit', 'cord', 'plenty', 'long', 'disconnected', 'middle', 'link', 'optional', 'sony', 'wired', 'remote', 'control', 'adapter', 'convert', 'standard', 'inch', 'stereo', 'jack', 'dual', 'inch', 'jacks', 'many', 'airplane', 'seats', 'included']
